In [0]:
data = [
    ("2022-01", "Produkt A", 50),
    ("2022-02", "Produkt B", 60),
    ("2022-03", "Produkt C", 70),
    ("2022-04", "Produkt D", 90),
    ("2022-05", "Produkt A", 20),
    ("2022-06", "Produkt B", 50),
    ("2022-07", "Produkt C", 30),
    ("2022-08", "Produkt A", 30),
    ("2022-09", "Produkt B", 10),
    ("2022-10", "Produkt C", 70),
    ("2022-11", "Produkt A", 90),
    ("2022-12", "Produkt B", 30)
]
df = spark.createDataFrame(data, ['date','product','sales'])
display(df)

date,product,sales
2022-01,Produkt A,50
2022-02,Produkt B,60
2022-03,Produkt C,70
2022-04,Produkt D,90
2022-05,Produkt A,20
2022-06,Produkt B,50
2022-07,Produkt C,30
2022-08,Produkt A,30
2022-09,Produkt B,10
2022-10,Produkt C,70


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window = Window.orderBy( col("date").desc() )
df1 = df.withColumn("row_number", row_number().over(window) )
display(df1)

date,product,sales,row_number
2022-12,Produkt B,30,1
2022-11,Produkt A,90,2
2022-10,Produkt C,70,3
2022-09,Produkt B,10,4
2022-08,Produkt A,30,5
2022-07,Produkt C,30,6
2022-06,Produkt B,50,7
2022-05,Produkt A,20,8
2022-04,Produkt D,90,9
2022-03,Produkt C,70,10


In [0]:
from pyspark.sql.functions import rank, desc

window = Window.orderBy( desc("sales") )

df1 = df.withColumn("sales_rank", rank().over(window) )
display(df1)

date,product,sales,sales_rank
2022-04,Produkt D,90,1
2022-11,Produkt A,90,1
2022-03,Produkt C,70,3
2022-10,Produkt C,70,3
2022-02,Produkt B,60,5
2022-01,Produkt A,50,6
2022-06,Produkt B,50,6
2022-07,Produkt C,30,8
2022-08,Produkt A,30,8
2022-12,Produkt B,30,8


In [0]:
from pyspark.sql.functions import dense_rank

window = Window.orderBy( desc("sales") )

df1 = df.withColumn("sales_rank_dense", dense_rank().over(window) )
display(df1)

date,product,sales,sales_rank_dense
2022-04,Produkt D,90,1
2022-11,Produkt A,90,1
2022-03,Produkt C,70,2
2022-10,Produkt C,70,2
2022-02,Produkt B,60,3
2022-01,Produkt A,50,4
2022-06,Produkt B,50,4
2022-07,Produkt C,30,5
2022-08,Produkt A,30,5
2022-12,Produkt B,30,5


In [0]:
from pyspark.sql.functions import percent_rank, asc

window = Window.orderBy( asc("sales") )

df1 = df.withColumn("sales_rank_percent", percent_rank().over(window) )
display(df1)

date,product,sales,sales_rank_percent
2022-09,Produkt B,10,0.0
2022-05,Produkt A,20,0.09090909090909091
2022-07,Produkt C,30,0.18181818181818182
2022-08,Produkt A,30,0.18181818181818182
2022-12,Produkt B,30,0.18181818181818182
2022-01,Produkt A,50,0.45454545454545453
2022-06,Produkt B,50,0.45454545454545453
2022-02,Produkt B,60,0.6363636363636364
2022-03,Produkt C,70,0.7272727272727273
2022-10,Produkt C,70,0.7272727272727273


In [0]:
from pyspark.sql.functions import lag, lead

window = Window.orderBy( asc("date") )

df1 = df.withColumn("prev_sales", lag(col("sales"), 1).over(window)  )
df1 = df1.withColumn("next_sales", lead(col("sales"), 1).over(window)  )
df1 = df1.withColumn("sales_change", (col("sales")-col("prev_sales"))/col('prev_sales') )
display(df1)


date,product,sales,prev_sales,next_sales,sales_change
2022-01,Produkt A,50,null,60,null
2022-02,Produkt B,60,50,70,0.2
2022-03,Produkt C,70,60,90,0.16666666666666666
2022-04,Produkt D,90,70,20,0.2857142857142857
2022-05,Produkt A,20,90,50,-0.7777777777777778
2022-06,Produkt B,50,20,30,1.5
2022-07,Produkt C,30,50,30,-0.4
2022-08,Produkt A,30,30,10,0.0
2022-09,Produkt B,10,30,70,-0.6666666666666666
2022-10,Produkt C,70,10,90,6.0


In [0]:
import pandas as pd
dfp = pd.read_csv("https://eurocash-pyspark.s3.eu-central-1.amazonaws.com/drinksbycountry.csv")
df = spark.createDataFrame(dfp).withColumnRenamed("total_litres_of_pure_alcohol", "total")

display(df)

country,beer_servings,spirit_servings,wine_servings,total,continent
Afghanistan,0,0,0,0.0,Asia
Albania,89,132,54,4.9,Europe
Algeria,25,0,14,0.7,Africa
Andorra,245,138,312,12.4,Europe
Angola,217,57,45,5.9,Africa
Antigua & Barbuda,102,128,45,4.9,North America
Argentina,193,25,221,8.3,South America
Armenia,21,179,11,3.8,Europe
Australia,261,72,212,10.4,Oceania
Austria,279,75,191,9.7,Europe


In [0]:
window = Window.partitionBy("continent").orderBy(col("total").desc())

df1 = df.withColumn("row_number", row_number().over(window) )\
    .filter( col("row_number")<=2 ).drop("row_number")
display(df1)

country,beer_servings,spirit_servings,wine_servings,total,continent
Nigeria,42,5,2,9.1,Africa
Gabon,347,98,59,8.9,Africa
Russian Federation,247,326,73,11.5,Asia
South Korea,140,16,9,9.8,Asia
Belarus,142,373,42,14.4,Europe
Lithuania,343,244,56,12.9,Europe
Grenada,199,438,28,11.9,North America
St. Lucia,171,315,71,10.1,North America
Australia,261,72,212,10.4,Oceania
New Zealand,203,79,175,9.3,Oceania


In [0]:
from pyspark.sql.functions import last

# Utwórz DataFrame
data = [("John", 10), ("Jane", None), ("Tom", 30), ("Bob", None), ("Alice", 50)]
df = spark.createDataFrame(data, ["Name", "Age"])
display(df)

# Utwórz specyfikację okna
window = Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Wypełnij wartości NaN ostatnimi wartościami nie będącymi NaN
df = df.withColumn("Age", last(df["Age"], ignorenulls=True).over(window))

# Wyświetl wynik
display(df)


Name,Age
John,10
Jane,null
Tom,30
Bob,null
Alice,50


Name,Age
John,10
Jane,10
Tom,30
Bob,30
Alice,50


In [0]:
from pyspark.sql.functions import first

# Utwórz DataFrame
data = [("John", 10), ("Jane", None), ("Tom", 30), ("Bob", None), ("Alice", 50)]
df = spark.createDataFrame(data, ["Name", "Age"])
display(df)

# Utwórz specyfikację okna
window = Window.rowsBetween(Window.currentRow, Window.unboundedFollowing)

# Wypełnij wartości NaN pierwszymi wartościami nie będącymi NaN
df = df.withColumn("Age", first(df["Age"], ignorenulls=True).over(window))

# Wyświetl wynik
display(df)


Name,Age
John,10
Jane,null
Tom,30
Bob,null
Alice,50


Name,Age
John,10
Jane,30
Tom,30
Bob,50
Alice,50


In [0]:
# średnia krocząca
from pyspark.sql.functions import avg

url = "https://stooq.pl/q/d/l/?s=pkn&d1=20230101&d2=20230619&i=d"
dfp = pd.read_csv(url)
df = spark.createDataFrame(dfp)

window = Window.orderBy("Data").rowsBetween(-5, 6)  # 11-punktowa średnia krocząca

# Obliczanie średniej kroczącej
df = df.withColumn("moving_average", avg("Zamkniecie").over(window))

display(df)

Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,moving_average
2023-01-02,64.24,65.0,63.9,64.6,572914,64.6
2023-01-03,64.72,65.82,64.6,65.2,1577473,64.9
2023-01-04,64.8,66.36,64.7,65.76,1827758,65.18666666666667
2023-01-05,65.2,65.62,61.92,62.46,5067874,64.505
2023-01-09,63.42,63.64,61.1,62.0,5678412,64.00399999999999
2023-01-10,61.64,62.54,61.28,62.0,4550427,63.669999999999995
2023-01-11,62.3,64.04,62.0,62.0,2974199,63.43142857142857
2023-01-12,62.64,63.38,62.3,63.12,2470506,63.3925
2023-01-13,63.3,63.38,62.38,62.5,1659283,63.29333333333333
2023-01-16,62.74,63.58,61.62,63.3,1622515,63.294
